In [2]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# from datetime import datetime
import datetime
from dateutil.relativedelta import relativedelta


load_dotenv()


hostname=os.getenv("hostname")
database=os.getenv("database")
username=os.getenv("username")
pwd=os.getenv("pwd")
port_id=os.getenv("port_id")
host = os.getenv("host")

import warnings
warnings.filterwarnings('ignore')

# xcrun: error: invalid active developer path

In [3]:
conn_string = f"postgresql://{username}:{pwd}@{host}:{port_id}/{database}"
db =  create_engine(url=conn_string)

In [4]:
transactions = pd.read_sql("SELECT * FROM transactions", db)
historicalPricesAG = pd.read_sql('SELECT * FROM "historical_pricesA_G" ', db)
historicalPricesHR = pd.read_sql('SELECT * FROM "historical_pricesH_R" ', db)
historicalPricesSZ = pd.read_sql('SELECT * FROM "historical_pricesS_Z" ', db)
senators = pd.read_sql("SELECT * FROM senator_info", db)

senators.head()

,row,senator,senatorId,party
0,1,Thomas H Tuberville,100,Republican
1,2,Thomas R Carper,101,Democratic
2,3,Daniel S Sullivan,102,Republican
3,4,Rick Scott,103,Republican
4,5,John Boozman,104,Republican


In [5]:
historicalPricesAG = pd.read_sql('SELECT * FROM "historical_pricesA_G" ', db)
historicalPricesHR = pd.read_sql('SELECT * FROM "historical_pricesH_R" ', db)
historicalPricesSZ = pd.read_sql('SELECT * FROM "historical_pricesS_Z" ', db)
senators = pd.read_sql("SELECT * FROM senator_info", db)

senators.head()

,row,senator,senatorId,party
0,1,Thomas H Tuberville,100,Republican
1,2,Thomas R Carper,101,Democratic
2,3,Daniel S Sullivan,102,Republican
3,4,Rick Scott,103,Republican
4,5,John Boozman,104,Republican


In [6]:
def extractDateFromTimestamp(row):
    return str(row).split(" ")[0]

In [7]:
transactions.columns[-1]

'alpha'

In [8]:
def returnSenatorNameFromId(senId):
    sens = senators.senator.to_list()
    senIds = senators.senatorId.to_list()

    senatorDict = dict(zip(senIds, sens))
    return senatorDict[senId]

returnSenatorNameFromId(103)

'Rick Scott'

In [9]:
def lookupIdByName(nameString):
    nameMatch = senators[senators['senator'].str.contains(nameString)]

    if len(nameMatch)>1:
        return nameMatch
    else: 
        return nameMatch['senatorId']


lookupIdByName("Tuber")

0    100
Name: senatorId, dtype: int64

In [10]:
tub  = transactions[transactions['senatorId']==100]
tub.ticker.value_counts().head(15)


--      50
ECOM    28
CLF     26
X       25
PYPL    22
OXY     18
INTC    17
SSYS    15
AA      13
MSFT    12
BABA     9
CALX     8
QCOM     8
ARKK     5
NU       5
Name: ticker, dtype: int64

In [11]:
tub[(tub['ticker']=='QCOM') & (tub['type']=='Purchase') 
& (tub['asset_type']=='Stock')
]


,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
5039,2022-06-16,Joint,QCOM,QUALCOMM Incorporated - Common Stock,Stock,Purchase,"$250,001 - $500,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/3...,07/13/2022,261,100,18.17
5217,2021-03-29,Joint,QCOM,QUALCOMM Incorporated,Stock,Purchase,"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/3...,07/23/2021,1002,100,1.19


In [12]:

def reformatDate(transDate):
    listDate  = transDate.split("/")
    return f"{listDate[2]}-{listDate[0]}-{listDate[1]}"

# transactions['transaction_date'] = transactions['transaction_date'].apply(lambda row: reformatDate(row))

# transactions['transaction_date'].head()

transactions.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
0,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,0,100,None
1,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,2,100,None
2,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,3,100,None
3,2022-10-06,Joint,XOM,Exxon Mobil Corporation Common Stock Option Ty...,Stock Option,Sale (Full),"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,8,100,None
4,2022-10-06,Joint,CVX,Chevron Corporation Common Stock Option Type: ...,Stock Option,Sale (Full),"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,9,100,None


In [13]:

testParams = (123, '2020-02-13', 'DLR')

In [14]:
for frame in [historicalPricesAG, historicalPricesHR, historicalPricesSZ]:
    frame['date'] = pd.to_datetime(frame['date'])

qc = historicalPricesHR[['QCOM', 'SPY', 'date']]
temp = qc[qc['date']>= '2021-03-29']
temp


,QCOM,SPY,date
0,120.2000,394.5900,2022-11-21
1,123.8500,396.0300,2022-11-18
2,121.4300,398.5100,2022-11-11
3,106.6900,376.3500,2022-11-04
4,119.2100,389.0200,2022-10-28
...,...,...,...
82,134.8904,408.3881,2021-04-30
83,131.6153,407.8400,2021-04-23
84,134.3170,408.3489,2021-04-16
85,136.6106,402.7022,2021-04-09


In [15]:
temp = temp.sort_values(by="date")
temp.head(10)

,QCOM,SPY,date
86,133.9089,392.0545,2021-04-01
85,136.6106,402.7022,2021-04-09
84,134.3170,408.3489,2021-04-16
83,131.6153,407.8400,2021-04-23
82,134.8904,408.3881,2021-04-30
81,133.9672,413.1052,2021-05-07
80,126.4841,407.6835,2021-05-14
79,127.7572,406.0785,2021-05-21
78,130.7504,411.0696,2021-05-28
77,131.2195,413.5749,2021-06-04


In [16]:


def createGrowthColumn(ticker, frame):

    pricesList = np.array(frame[ticker])
    pctChanges = np.diff(pricesList)/pricesList[:-1]

    currentInvestment , initialInvestment= 100, 100
    investmentValues = []

    for c in pctChanges[1:]: 
        currentInvestment = currentInvestment*(1+c) 
        investmentValues.append(currentInvestment)

    pctChanges = [np.nan ] + list(pctChanges)
    investmentValues = [ np.nan, initialInvestment ] + investmentValues

    invName= ticker+'_growth'
    pctName = ticker+'_pchange'
    frame[invName] = investmentValues
    frame[pctName] = pctChanges



createGrowthColumn('QCOM', temp)
createGrowthColumn('SPY', temp)
temp.head()


,QCOM,SPY,date,QCOM_growth,QCOM_pchange,SPY_growth,SPY_pchange
86,133.9089,392.0545,2021-04-01,NaN,NaN,NaN,NaN
85,136.6106,402.7022,2021-04-09,100.000000,0.020176,100.000000,0.027159
84,134.3170,408.3489,2021-04-16,98.321067,-0.016789,101.402202,0.014022
83,131.6153,407.8400,2021-04-23,96.343402,-0.020114,101.275831,-0.001246
82,134.8904,408.3881,2021-04-30,98.740800,0.024884,101.411937,0.001344


In [17]:
import plotly.express as px 

fig = px.line(temp, x='date', y=['QCOM_growth', 'SPY_growth' ],
 title="Growth in Qualcomm v S&P"
 )


fig.show()

What a great example! looks like there was a sharp rise in price starting in November, and by mid January Qualcomm had grown much more than the S&P. My hypothesis is that the majority of stock purchases will exhibit a similar pattern

In [18]:
import string 
print(string.ascii_uppercase)

ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [19]:
def findSourceFrame(ticker):
    leadingLetter =ticker[0]

    relevantCols = ['date', ticker, 'SPY']

    if leadingLetter in 'ABCDEFG':
        rez = historicalPricesAG[relevantCols]
    elif leadingLetter in 'HIJKLMNOPQR':
        rez = historicalPricesHR[relevantCols]
    elif leadingLetter in 'STUVWXYZ':
        rez = historicalPricesSZ[relevantCols]
    else:
        return 'ticker leading letter is non-alpha'
    return rez


In [20]:


def findBetweenPrice(col, frame, currentDate):

    # check if its a friday 
    dateInputFormatted = datetime.datetime.strptime(currentDate, "%Y-%m-%d")

    isFriday = (dateInputFormatted.weekday()+3)%7==0

    # if its a friday, return the friday price (do nothing)

    if isFriday: 
        fridayPrice = frame[frame['date']==currentDate][col].values[0]
        return fridayPrice


    newFrame = frame.copy()

    newFrame['date'] = pd.to_datetime(newFrame['date'])

    dateList = newFrame['date'].to_list()

    datesBeforeCurrent = list(filter(lambda d: d < dateInputFormatted, dateList))
    datesAfterCurrent = list(filter(lambda d: d > dateInputFormatted, dateList))

    previousDateAvailable = datesBeforeCurrent[-1]

 

    previousPrice = newFrame[newFrame['date']==previousDateAvailable][col].values[0]


    # return datesAfterCurrent
    if len(datesAfterCurrent)==0:
        return previousPrice
        
    nextDateAvailable = datesAfterCurrent[0]

    nextPrice = newFrame[newFrame['date']==nextDateAvailable][col].values[0]

  

    daysSinceLastAvailable  = dateInputFormatted  - previousDateAvailable

    daysSinceLastAvailable = daysSinceLastAvailable.days

    daysBetweenAvailableDates = nextDateAvailable - previousDateAvailable

    daysBetweenAvailableDates =  daysBetweenAvailableDates.days



    # priceDiff =  previousPrice - nextPrice
    priceDiff =  nextPrice - previousPrice

    dailyPriceDiff = priceDiff/daysBetweenAvailableDates

    newStartingPrice = previousPrice+(dailyPriceDiff*daysSinceLastAvailable) 

    return newStartingPrice


In [21]:
def getStarterFrame(ticker, dateInput):
    rez = findSourceFrame(ticker)

    rez = rez.sort_values(by="date")

    prevWeekDate = datetime.datetime.strptime(dateInput, "%Y-%m-%d") + datetime.timedelta(weeks=-1)

    # start dataframe at purchase date 
    rez = rez[rez['date']>=prevWeekDate]

    # end dataframe at one year from purchase date 
    if len(rez)>52:
        rez = rez.head(53)

    return rez

# starterFrame = getStarterFrame("REGN", '2021-03-29')
starterFrame = getStarterFrame("A", '2021-03-29')
starterFrame.head()



,date,A,SPY
87,2021-03-26,124.3421,387.5234
86,2021-04-01,126.3709,392.0545
85,2021-04-09,130.7421,402.7022
84,2021-04-16,133.3885,408.3489
83,2021-04-23,135.4699,407.8400


In [22]:

# starterFrame.date[0]
starterDates = starterFrame.date.to_list()
testDate = starterDates[0]

# testDate < datetime.datetime.strptime('2020-01-01', "%Y-%m-%d")


# list(starterDates.filter(lambda d: d< datetime.datetime.strptime('2020-01-01', "%Y-%m-%d")))
# list(filter(lambda d: d< datetime.datetime.strptime('2021-04-01', "%Y-%m-%d"), starterDates))[-1]
list(filter(lambda d: d > datetime.datetime.strptime('2021-03-29', "%Y-%m-%d"), starterDates))[0]


Timestamp('2021-04-01 00:00:00')

In [23]:
# starterDates
testDate



Timestamp('2021-03-26 00:00:00')

In [24]:

def generateHistoricalPricesTable(ticker, dateInput):
    
    rez = findSourceFrame(ticker)

    rez = rez.sort_values(by="date")

    prevWeekDate = datetime.datetime.strptime(dateInput, "%Y-%m-%d") + datetime.timedelta(weeks=-1)

    # start dataframe at purchase date 
    rez = rez[rez['date']>=prevWeekDate]
    
    # end dataframe at one year from purchase date 
    if len(rez)>52:
        rez = rez.head(53)
    
    # rez = rez.append(newRow, ignore_index=True)

    rez['date'] = rez['date'].apply(lambda row: extractDateFromTimestamp(row))

    rez = rez.sort_values(by="date")

    return rez


graphDims = {'height':500,  'width':800}

def graphGrowthRates(frame, tickerFirst, tickerSecond='SPY'):
    yColumnList = [tickerFirst+"_growth", tickerSecond+"_growth"]
    fig = px.line(frame, x='date', y=yColumnList,
    title=f"Growth in {tickerFirst} v {tickerSecond}",
        height= graphDims['height'] , width= graphDims['width']
    )
    fig.show()


# putting it all together 
def graphAlpha(ticker, date):

    histPrices =  generateHistoricalPricesTable(ticker, date)
    createGrowthColumn(ticker, histPrices)
    createGrowthColumn('SPY', histPrices)

    graphGrowthRates(histPrices, ticker)


# generateHistoricalPricesTable("REGN", '2021-03-29').head()
# graphAlpha("REGN", '2021-03-29')
generateHistoricalPricesTable("DLR", "2020-02-12").head()


,date,DLR,SPY
146,2020-02-07,112.3515,317.9901
145,2020-02-14,119.8428,323.1592
144,2020-02-21,125.8851,319.2154
143,2020-02-28,109.4625,283.5875
142,2020-03-06,116.8718,284.7361


In [25]:

# generateHistoricalPricesTable('DLR', '2020-02-13')
# graphAlpha('DLR', '2020-02-13')
# graphAlpha('DLR', '2020-02-13')

dateInputFormatted = datetime.datetime.strptime('2020-02-13', "%Y-%m-%d")

dateInputFormatted.weekday()

3

In [26]:

def purchasesBySenator(senId):
    isSenator = transactions['senatorId']==senId
    isPurchase = transactions['type']=='Purchase'
    isStock = transactions['asset_type']=='Stock'
    rez = transactions[isSenator & isPurchase & isStock]

    return rez.loc[:, :'type']


tbPurchases = purchasesBySenator(100)
tbPurchases.tail(10)


,transaction_date,owner,ticker,asset_description,asset_type,type
5234,2021-03-29,Joint,ATHM,Autohome Inc.,Stock,Purchase
5235,2021-03-29,Joint,AME,"AMETEK, Inc.",Stock,Purchase
5236,2021-03-29,Joint,GOOGL,Alphabet Inc.,Stock,Purchase
5237,2021-03-16,Joint,AA,Alcoa Corporation,Stock,Purchase
5238,2021-03-16,Joint,AA,Alcoa Corporation,Stock,Purchase
5239,2021-03-12,Joint,CLF,Cleveland-Cliffs Inc.,Stock,Purchase
5240,2021-04-19,Joint,CLF,Cleveland-Cliffs Inc.,Stock,Purchase
5241,2021-04-16,Joint,ECOM,ChannelAdvisor Corporation,Stock,Purchase
5243,2021-04-07,Joint,OXY,Occidental Petroleum Corporation,Stock,Purchase
5244,2021-05-06,Joint,X,United States Steel Corporation,Stock,Purchase


In [27]:
graphAlpha("OXY", '2021-06-25')

In [28]:
alDate = reformatDate("03/16/2021")
graphAlpha('AA', alDate)


Alright so Tommy Tuberville had a great year in 2021, but the S&P was pretty lousy that year. Let's try a different Senator and another time period

In [29]:
senSample = senators.sample(n=30, random_state=69)['senatorId'].to_list()
# lets find the senator with the most purchases within a random sample of 30 senators 
maxPurchases= 0
maxSenator=0
for sen in senSample:
    currentPurchases = len(purchasesBySenator(sen))
    if currentPurchases >= maxPurchases:
        maxPurchases = currentPurchases
        maxSenator = sen

print(senators[senators['senatorId']==maxSenator])
purchasesBySenator(maxSenator).tail(10)

    row              senator  senatorId       party
34   35  David A Perdue , Jr        136  Republican


,transaction_date,owner,ticker,asset_description,asset_type,type
8259,2015-04-17,Joint,KN,Knowles Corporation (NYSE),Stock,Purchase
8260,2015-04-23,Joint,KN,Knowles Corporation (NYSE),Stock,Purchase
8261,2015-04-24,Joint,KN,Knowles Corporation (NYSE),Stock,Purchase
8262,2015-04-24,Joint,KN,Knowles Corporation (NYSE),Stock,Purchase
8263,2015-04-15,Joint,MOS,The Mosaic Company (NYSE),Stock,Purchase
8264,2015-05-12,Joint,DISCA,"Discovery Communications, Inc. (NASDAQ)",Stock,Purchase
8265,2015-05-13,Joint,DISCA,"Discovery Communications, Inc. (NASDAQ)",Stock,Purchase
8266,2015-05-14,Joint,DISCA,"Discovery Communications, Inc. (NASDAQ)",Stock,Purchase
8267,2015-05-15,Joint,AXLL,Axiall Corporation (NYSE),Stock,Purchase
8268,2015-05-29,Joint,DISCA,"Discovery Communications, Inc. (NASDAQ)",Stock,Purchase


In [30]:
graphAlpha("MOS", "2015-04-15")
print("this is is kind of a terrible trade")

this is is kind of a terrible trade


In [31]:
graphAlpha("KN", "2015-04-24")
print("this one has a bit of a pop, but also not a great trade")

this one has a bit of a pop, but also not a great trade


In [32]:
purchasesBySenator(136).iloc[-60:-50, :]

,transaction_date,owner,ticker,asset_description,asset_type,type
8209,2015-04-02,Joint,FOX,"Twenty-First Century Fox, Inc. (NASDAQ)",Stock,Purchase
8210,2015-04-02,Joint,AVP,Avon Products Inc. (NYSE),Stock,Purchase
8211,2015-04-02,Joint,JNJ,Johnson & Johnson (NYSE),Stock,Purchase
8212,2015-04-02,Joint,KO,The Coca-Cola Company (NYSE),Stock,Purchase
8213,2015-05-06,Joint,SYY,Sysco Corporation (NYSE),Stock,Purchase
8214,2015-05-11,Joint,FOX,"Twenty-First Century Fox, Inc. (NASDAQ)",Stock,Purchase
8215,2015-05-12,Joint,FOX,"Twenty-First Century Fox, Inc. (NASDAQ)",Stock,Purchase
8216,2015-01-07,Joint,AM,Antero Midstream Partners LP (NYSE),Stock,Purchase
8217,2015-01-14,Joint,AR,Antero Resources Corporation (NYSE),Stock,Purchase
8218,2015-01-15,Joint,BPL,"Buckeye Partners, L.P. (NYSE)",Stock,Purchase


In [33]:
graphAlpha("AM", "2015-01-15")

In [34]:
# purchasesBySenator(136).iloc[-90:-80, :]
purchasesBySenator(136).iloc[20:30]

,transaction_date,owner,ticker,asset_description,asset_type,type
5676,2020-03-06,Joint,BRK-B,Berkshire Hathaway Inc.,Stock,Purchase
5677,2020-03-06,Joint,BRK-B,Berkshire Hathaway Inc.,Stock,Purchase
5678,2020-03-09,Joint,BRK-B,Berkshire Hathaway Inc.,Stock,Purchase
5679,2020-03-10,Joint,MSM,"MSC Industrial Direct Co., Inc.",Stock,Purchase
5680,2020-03-10,Joint,M,"Macy's, Inc.",Stock,Purchase
5681,2020-03-10,Joint,NWSA,News Corporation,Stock,Purchase
5682,2020-03-10,Joint,FOXA,Fox Corporation,Stock,Purchase
5683,2020-03-10,Joint,EAF,GrafTech International Ltd.,Stock,Purchase
5684,2020-03-13,Joint,ENB,Enbridge Inc.,Stock,Purchase
5685,2020-03-13,Joint,TRP,TC Energy Corporation,Stock,Purchase


In [35]:
graphAlpha("M", "2020-03-10")

here's a great example of why we need more information. Its true Macy's had a big run, but this run was 6 months after the purchase, and it was preceeded by a smaller run. Hard to say if the first run was the reason for the purcahse or not. Lets modify the graph alpha function to indicate all transacions (purcahses and sales) of one asset by a senator within the measurement window (1 year)

In [36]:
from dateutil.relativedelta import  relativedelta

transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])


def findRelevantTransactions(senId, date, ticker):
    # startDate = pd.to_datetime(date) + relativedelta(days=-1)
    startDate = pd.to_datetime(date)
    endDate = startDate + relativedelta(years=1)
    
    afterStartDate = transactions['transaction_date']>= startDate
    beforeEndDate = transactions['transaction_date']< endDate
    isStock = transactions['asset_type']=="Stock"
    isSenator = transactions['senatorId']==senId
    isTicker = transactions['ticker']==ticker
    # rez  = transactions[afterStartDate & beforeEndDate & isStock & isSenator  ]

    rez = transactions[afterStartDate & beforeEndDate & isStock & isSenator & isTicker]

    return rez 



print("looks like this senator bought then sold a few weeks later, thereby missing the second Macy's second big run ")

macysTransactions = findRelevantTransactions(136, "2020-03-10", "M")[['transaction_date', 'type','amount', 'senator', 'ticker']]
macysTransactions

looks like this senator bought then sold a few weeks later, thereby missing the second Macy's second big run 


,transaction_date,type,amount,senator,ticker
1017,2020-04-14,Sale (Full),"$15,001 - $50,000","David A Perdue , Jr",M
5680,2020-03-10,Purchase,"$1,001 - $15,000","David A Perdue , Jr",M
5718,2020-03-20,Purchase,"$1,001 - $15,000","David A Perdue , Jr",M


In [37]:

#generate map of dates and transaction types 
dates = macysTransactions['transaction_date'].to_list()
transTypes = macysTransactions['type'].to_list()

#calculate the closest friday to merge with historical prices table (weekly data)
def nearestFriday(inputDate):
    fridayDelta = datetime.timedelta( (4-inputDate.weekday()) % 7 )
    nearestFriday = inputDate + fridayDelta
    nearestFridayString = datetime.datetime.strftime(nearestFriday , "%Y-%m-%d")
    return nearestFridayString

fridayDates  = [nearestFriday(d) for d in dates]

transactionMap = dict(zip(fridayDates, transTypes))



transactionMap 

{'2020-04-17': 'Sale (Full)',
 '2020-03-13': 'Purchase',
 '2020-03-20': 'Purchase'}

In [38]:

# putting it all together
# find relevant transactions
def historicalPricesWithTransaction(senatorIdInput, dateInput,tickerInput ):
    relevantTransactions = findRelevantTransactions(senatorIdInput, dateInput,tickerInput)
    # return relevantTransactions

    relevantTransactionDates = relevantTransactions['transaction_date'].to_list()
    relevantTransactionTypes = relevantTransactions['type'].to_list()

    # chop off first date
    hist = generateHistoricalPricesTable(tickerInput, dateInput)

    hist['transactionType'] = np.nan
    hist['transactionValue'] = np.nan 

    # return relevantTransactions


    for index, d in enumerate(relevantTransactionDates):

        d = extractDateFromTimestamp(d)
        tickerPurchasePrice =  findBetweenPrice(tickerInput, hist, d)
        spyPurchasePrice = findBetweenPrice('SPY', hist, d)

        currentTransType = relevantTransactionTypes[index]

       
    
    
        newRow = {'date': d, tickerInput: tickerPurchasePrice, 'SPY': spyPurchasePrice, 
        'transactionType': currentTransType, 'transactionValue': tickerPurchasePrice}

        hist = hist.append(newRow, ignore_index=True)


    # return relevantTransactionDates, relevantTransactionTypes

    hist = hist.sort_values(by="date")

    # hist = hist.drop_duplicates(subset="date", keep=False)
    return hist




# temp = historicalPricesWithTransaction(136,  "2020-03-10", "M" )

# temp = historicalPricesWithTransaction(100, '2021-03-29',"REGN")
# historicalPricesWithTransaction(110, '2020-02-14', 'DLR')

# temp.head(10)
# temp = historicalPricesWithTransaction(101, '2012-12-17','DD')
# temp



In [39]:
# testParams (123, '2020-02-13', 'DLR')

# transactions[transactions['transaction_date']=='2020-02-13'].ticker.value_counts().index[-1]
# transactions[transactions[transactions['transaction_date']=='2020-02-13']['ticker']=='DLR']
# transactions[transactions['transaction_date']=='2020-02-13']
findRelevantTransactions(*testParams)


,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
5761,2020-02-13,Self,DLR,"Digital Realty Trust, Inc.",Stock,Purchase,"$15,001 - $50,000",--,Sheldon Whitehouse,https://efdsearch.senate.gov/search/view/ptr/2...,03/31/2020,2554,123,31.68


In [40]:
# temp = historicalPricesWithTransaction(125,  "2017-12-21", "WTW" )
temp = historicalPricesWithTransaction(136,  "2020-03-10", "M" )
temp.head()

,date,M,SPY,transactionType,transactionValue
0,2020-03-06,10.542300,284.736100,NaN,NaN
54,2020-03-10,8.943557,269.343929,Purchase,8.943557
1,2020-03-13,7.744500,257.799800,NaN,NaN
55,2020-03-20,5.806000,220.358500,Purchase,5.806000
2,2020-03-20,5.806000,220.358500,NaN,NaN


In [41]:
import plotly.graph_objects as go 

def graphTransactions_fromHistorical(senatorIdInput, frame, tickerInput):


    senatorName = returnSenatorNameFromId(senatorIdInput)


    fig = px.scatter(frame, x="date", y="transactionValue", color="transactionType",
    labels=dict(transactionValue="historical Price "+ tickerInput),
    height= graphDims['height'] , width= graphDims['width'], 
    title = senatorName+ " Historical Price and Transactions "+ tickerInput)

    fig.add_trace(go.Scatter(x=frame['date'], y=frame[tickerInput], mode="lines", line=go.scatter.Line(color="grey"), 
    name=tickerInput+' Historical Price'))



    fig.show()

graphTransactions_fromHistorical(136, temp, "M")



this wasn't the best series of trades. Next let's combine the preceeding couple of functions to examine trade's success by only inputting the transaction date, ticker, and senator. 

In [42]:
# putting it all together 

def graphTransactions_fromSource(senatorIdInput, dateInput,tickerInput):
    historicalPricesFrame = historicalPricesWithTransaction(senatorIdInput, dateInput,tickerInput )

    graphTransactions_fromHistorical(senatorIdInput, historicalPricesFrame, tickerInput)



In [43]:

purchases = transactions[(transactions['type']=='Purchase') & (transactions['asset_type']=='Stock') ]
randomPurchase = purchases.sample(n=1, random_state=227)[['transaction_date','ticker','asset_description','senator','senatorId']]
randomPurchase

,transaction_date,ticker,asset_description,senator,senatorId
7706,2016-03-31,WMT,Wal-Mart Stores Inc.,William Cassidy,134


In [44]:
# randomPurchase.ticker[0]
randomticker = randomPurchase.ticker.values[0]
randomDateFull = randomPurchase.transaction_date.values[0]
randomDate = str(randomDateFull).split("T")[0]
randomSenator = randomPurchase.senatorId.values[0]

graphTransactions_fromSource(randomSenator,  randomDate, randomticker)


Looks like Kelly Loeffler's purchase had a pretty big pop after she bought it, but she sold and missed out on a lot of gains. 
Let's graph some more transactions at random

In [45]:
def generateRandomPurchase(randomState):
    purchases = transactions[transactions['type']=='Purchase']
    randomPurchase = purchases.sample(n=1,random_state=randomState)[['transaction_date','ticker','asset_description','senator','senatorId']]


    print(randomPurchase)
    randomticker = randomPurchase.ticker.values[0]
    randomDateFull = randomPurchase.transaction_date.values[0]
    randomDate = str(randomDateFull).split("T")[0]
    randomSenator = randomPurchase.senatorId.values[0]

    return randomSenator,  randomDate, randomticker


def graphRandomPurcahse(randomState):

    randomSenator,  randomDate, randomticker = generateRandomPurchase(randomState)

    graphTransactions_fromSource(randomSenator,  randomDate, randomticker)


graphRandomPurcahse(427)



     transaction_date ticker asset_description              senator  senatorId
5726       2020-03-24  BOIVF           Bollore  David A Perdue , Jr        136


Excellent trades David Perdue! 

In [46]:
graphRandomPurcahse(430)

     transaction_date ticker              asset_description          senator  \
5323       2020-11-20   MRVL  Marvell Technology Group Ltd.  Thomas R Carper   

      senatorId  
5323        101  


Well done Ronald! Personally, I wouldn't sell either. There's a substantial jump in price, particularly after the first purchase. However, this analsysis is only sound if we differentiate between:  <br>
    &nbsp; changes in the marketplace in general AND <br>
    &nbsp; changes in this stock in particular <br>
Next we build upon previous funcions, graphing all transactions of a single senator in a single stock, but using growth rates instead of just historical prices. 

In [47]:
def historicalGrowthRatesWithTransactions(senatorIdInput, dateInput,tickerInput ):



    #start dataframe a week early so growth rates begin at 1
    prevWeekDate = datetime.datetime.strptime(dateInput, "%Y-%m-%d") +datetime.timedelta(weeks=-1)
    prevWeekString = datetime.datetime.strftime(prevWeekDate,"%Y-%m-%d")
    
    historicalPricesFrame = historicalPricesWithTransaction(senatorIdInput, dateInput,tickerInput )

    createGrowthColumn(tickerInput, historicalPricesFrame)
    createGrowthColumn('SPY', historicalPricesFrame)

    growthColumn = tickerInput+'_growth'


    return historicalPricesFrame



# temp = historicalGrowthRatesWithTransactions(136,  "2020-03-10", "M"  )
# temp = historicalGrowthRatesWithTransactions(*testParams)
temp = historicalGrowthRatesWithTransactions(100, '2021-03-29',"REGN")



temp.head()

,date,REGN,SPY,transactionType,transactionValue,REGN_growth,REGN_pchange,SPY_growth,SPY_pchange
0,2021-03-26,476.91,387.52340,NaN,NaN,NaN,NaN,NaN,NaN
53,2021-03-29,475.92,389.78895,Purchase,475.92,100.000000,-0.002076,100.000000,0.005846
1,2021-04-01,474.93,392.05450,NaN,NaN,99.791982,-0.002080,100.581225,0.005812
2,2021-04-09,475.17,402.70220,NaN,NaN,99.842410,0.000505,103.312883,0.027159
3,2021-04-16,502.60,408.34890,NaN,NaN,105.605984,0.057727,104.761538,0.014022


In [48]:
# temp = historicalGrowthRatesWithTransactions(123,  "2020-02-13", "DLR"  )
temp = historicalGrowthRatesWithTransactions(*testParams )

temp.head()

,date,DLR,SPY,transactionType,transactionValue,DLR_growth,DLR_pchange,SPY_growth,SPY_pchange
0,2020-02-07,112.351500,317.990100,NaN,NaN,NaN,NaN,NaN,NaN
53,2020-02-13,118.772614,322.420757,Purchase,118.772614,100.000000,0.057152,100.000000,0.013933
1,2020-02-14,119.842800,323.159200,NaN,NaN,100.901037,0.009010,100.229031,0.002290
2,2020-02-21,125.885100,319.215400,NaN,NaN,105.988321,0.050419,99.005847,-0.012204
3,2020-02-28,109.462500,283.587500,NaN,NaN,92.161397,-0.130457,87.955721,-0.111611


In [49]:
temp = historicalGrowthRatesWithTransactions(*testParams)

def graphAlphaGrowth_fromHistorical(senatorIdInput, frame, tickerInput):
    senatorName = returnSenatorNameFromId(senatorIdInput)

    growthColumn = tickerInput+'_growth'


    fig = px.scatter(frame, x="date", 
    # y="transactionValue", 
    y=growthColumn,
    color="transactionType",
    labels=dict(transactionValue="Growth Rate"),
    height= graphDims['height'] , width= graphDims['width'], 
    title = senatorName+ " Growth Rates "+ tickerInput)



    fig.add_trace(go.Scatter(x=frame['date'], y=frame[growthColumn], mode="lines", line=go.scatter.Line(color="#EB4E4B"), 
    name=tickerInput+' Growth Rate'))

    fig.add_trace(go.Scatter(x=frame['date'], y=frame['SPY_growth'], mode="lines", line=go.scatter.Line(color="#349FEB"), 
    name='SPY Growth Rate'))



    fig.show()

graphAlphaGrowth_fromHistorical(123, temp, 'DLR')
# graphAlphaGrowth_fromHistorical(136, temp, "M")

In [50]:
def graphAlphaGrowth_fromSource(senatorIdInput, dateInput,tickerInput):
    historical = historicalGrowthRatesWithTransactions(senatorIdInput, dateInput,tickerInput )

    # historical = historical.iloc[1:, :]
    # return historical
    graphAlphaGrowth_fromHistorical(senatorIdInput, historical , tickerInput)

In [51]:
# historicalGrowthRatesWithTransactions(100, '2021-03-29','REGN')
# graphAlphaGrowth_fromSource(*testParams)/
graphAlphaGrowth_fromSource(100, '2021-03-29','REGN')

In [52]:

# tempAgain = historicalGrowthRatesWithTransactions(*testParams)
# tempAgain

In [53]:
def graphRandomPurcahse_alpha(randomState):
    randomSenator,  randomDate, randomticker = generateRandomPurchase(randomState)

    graphAlphaGrowth_fromSource(randomSenator,  randomDate, randomticker)

graphRandomPurcahse_alpha(765)

     transaction_date ticker        asset_description         senator  \
5753       2020-03-02     DD  DuPont de Nemours, Inc.  Kelly Loeffler   

      senatorId  
5753        137  


# Finding Measurement Window


In [54]:
randomSenator,  randomDate, randomticker = generateRandomPurchase(456)
findRelevantTransactions(randomSenator,  randomDate, randomticker)

     transaction_date ticker asset_description              senator  senatorId
5566       2020-04-06    ENB     Enbridge Inc.  David A Perdue , Jr        136


,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
1042,2020-04-15,Joint,ENB,Enbridge Inc.,Stock,Sale (Full),"$1,001 - $15,000",--,"David A Perdue , Jr",https://efdsearch.senate.gov/search/view/ptr/b...,2020-05-08,2004,136,None
5566,2020-04-06,Joint,ENB,Enbridge Inc.,Stock,Purchase,"$1,001 - $15,000",--,"David A Perdue , Jr",https://efdsearch.senate.gov/search/view/ptr/b...,05/08/2020,1902,136,3.37


In [55]:
purchases.sort_values("transaction_date").head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
8295,2012-09-13,Spouse,DD,E. I. du Pont de Nemours and Company (NYSE),Stock,Purchase,"$1,001 - $15,000",--,Thomas R Carper,https://efdsearch.senate.gov/search/view/ptr/6...,05/13/2015,8458,101,0.18
8294,2012-12-17,Spouse,DD,E. I. du Pont de Nemours and Company (NYSE),Stock,Purchase,"$1,001 - $15,000",--,Thomas R Carper,https://efdsearch.senate.gov/search/view/ptr/6...,05/13/2015,8457,101,11.93
8293,2013-03-15,Spouse,DD,E. I. du Pont de Nemours and Company (NYSE),Stock,Purchase,"$1,001 - $15,000",--,Thomas R Carper,https://efdsearch.senate.gov/search/view/ptr/6...,05/13/2015,8456,101,Ticker Not Found
8292,2013-06-13,Spouse,DD,E. I. du Pont de Nemours and Company (NYSE),Stock,Purchase,"$1,001 - $15,000",--,Thomas R Carper,https://efdsearch.senate.gov/search/view/ptr/6...,05/13/2015,8455,101,8.74
8291,2013-09-13,Spouse,DD,E. I. du Pont de Nemours and Company (NYSE),Stock,Purchase,"$1,001 - $15,000",--,Thomas R Carper,https://efdsearch.senate.gov/search/view/ptr/6...,05/13/2015,8454,101,3.64


In [56]:
graphAlphaGrowth_fromSource(101, '2012-09-13','DD')


Observe the multiple purcahses of DD made by Thomas Carper in 2012. His first purchase in September did not result in a growth rate more than the market average. 

In [57]:
graphAlphaGrowth_fromSource(101, '2012-12-17','DD')

Starting the clock from his next purcahse in December of 2012 paints a rosier picture. From here, he managed to beat the S&P by a wide margin.  Thomas' first purcahse was poorly timed, but is subsequent purcahse three months later was very profitable. This is why every purcahse needs to be examined separately.
<br>
<br>
the analysis that follows will calculate alpha as the maximum difference in growth indecies between a stock and the S&P 6 months after every purcahse. 

In [58]:
def alphaFrame(senatorIdInput, dateInput,tickerInput ):
    hist = historicalGrowthRatesWithTransactions(senatorIdInput, dateInput,tickerInput)
    # hist.drop(['transactionType', 'transactionValue'], axis=1, inplace=True)

    sixMonthsHence = datetime.datetime.strptime(dateInput, "%Y-%m-%d") + relativedelta(months=+6)

    hist['date'] = pd.to_datetime(hist['date'])


    hist = hist[hist['date']<=sixMonthsHence]

    growthCol = f"{tickerInput}_growth"
    #calculate alpha as difference in growth rates 
    hist['alpha']  = hist[growthCol] - hist['SPY_growth']
    return hist

# temp = alphaFrame(101, '2012-12-21','DD')
# temp = alphaFrame(101, '2012-12-21','M')
temp = alphaFrame(101, '2012-12-21','F')
# temp = alphaFrame(*testParams)
temp.head()

,date,F,SPY,transactionType,transactionValue,F_growth,F_pchange,SPY_growth,SPY_pchange,alpha
519,2012-12-14,7.3834,117.6348,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,2012-12-21,7.8890,119.0519,NaN,NaN,100.000000,0.068478,100.000000,0.012047,0.000000
517,2012-12-28,8.5608,116.7507,NaN,NaN,108.515655,0.085157,98.067062,-0.019329,10.448593
516,2013-01-04,9.0264,122.0367,NaN,NaN,114.417543,0.054387,102.507142,0.045276,11.910402
515,2013-01-11,9.3124,122.6204,NaN,NaN,118.042844,0.031685,102.997432,0.004783,15.045412


In [59]:
def calculateAlphaValue(senatorIdInput, dateInput,tickerInput ):

    try:         
        # we're looking to return the maximum alpha value within a 6 month period
        histAlpha = alphaFrame(senatorIdInput, dateInput,tickerInput )

    except: 
        return 'Not Found'

        # start counting a week after purcahse
    histAlpha = histAlpha.iloc[ 1:,:]

    maxAlpha = histAlpha.loc[histAlpha.alpha.idxmax()].alpha
    maxAlpha = round(maxAlpha, 2)

    return maxAlpha



In [60]:
purchases.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
28,2020-11-17,Joint,W,Wayfair Inc.,Stock,Purchase,"$15,001 - $50,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/b...,12/03/2020,1338,133,28.15
85,2020-08-21,Spouse,SQ,"Square, Inc.",Stock,Purchase,"$15,001 - $50,000",--,Ron L Wyden,https://efdsearch.senate.gov/search/view/ptr/3...,08/29/2020,1565,163,62.47
4975,2020-08-24,Spouse,SQ,"Square, Inc.",Stock,Purchase,"$15,001 - $50,000",--,Ron L Wyden,https://efdsearch.senate.gov/search/view/ptr/2...,09/24/2020,1481,163,63.66
4976,2017-03-16,Spouse,SNY,Sanofi,Stock,Purchase,"$1,001 - $15,000",--,Patty Murray,https://efdsearch.senate.gov/search/view/ptr/6...,03/21/2017,6357,156,14.67
4977,2020-11-20,Spouse,V,Visa Inc.,Stock,Purchase,"$1,001 - $15,000",--,Thomas R Carper,https://efdsearch.senate.gov/search/view/ptr/3...,12/02/2020,1349,101,1.61


In [61]:
alphaList = []

for row in list(np.arange(len(purchases))):
    currentDate = purchases.transaction_date.values[row]
    currentTicker = purchases.ticker.values[row]
    currentSenId = purchases.senatorId.values[row]


    currentDateString  = str(currentDate).split("T")[0]
   
    try:
        currentAlpha = calculateAlphaValue(currentSenId, currentDateString,currentTicker)
    except:
        currentAlpha ="Not Found"


    

    alphaList.append(currentAlpha)
alphaList[:5]


[28.15, 62.47, 63.66, 14.67, 1.61]

In [62]:
purchases['alpha'] = alphaList
# purchases.sort_values(by="alpha", ascending=False)
# purchases.head()
purchases['alpha'] = purchases['alpha'].replace("Not Found", -9999)
topAlphaBuys = purchases.sort_values(by="alpha", ascending=False).head(15)
topAlphaBuys[['transaction_date', 'ticker','senatorId','asset_description', 'alpha']]


,transaction_date,ticker,senatorId,asset_description,alpha
5344,2020-11-20,SI,101,Silvergate Capital Corporation,450.03
6846,2017-10-19,CRSP,132,CRISPR Therapeutics AG,189.74
5404,2020-08-03,TSLA,133,"Tesla, Inc.",189.02
5338,2020-11-20,NTLA,101,"Intellia Therapeutics, Inc.",147.03
5443,2020-07-07,TSLA,118,"Tesla, Inc.",129.25
7654,2016-05-18,NVDA,123,NVIDIA Corporation,108.65
7840,2016-01-08,CLR,132,"Continental Resources, Inc.",98.27
8314,2015-04-07,NFLX,123,"Netflix, Inc. (NASDAQ)",96.67
5303,2020-12-21,EXPI,133,"eXp World Holdings, Inc.",94.13
5299,2020-12-22,PLUG,133,Plug Power Inc.,91.43


There you have it, the best stock purchases made by senators. Lets graph a few of these to see what they look like

In [63]:
graphAlphaGrowth_fromSource(101, '2020-11-20','SI')

In [64]:
graphAlphaGrowth_fromSource(132, '2017-10-19','CRSP')

# Alpha EDA

In [65]:
#remove fill in values
purchasesFilt = purchases[ purchases['alpha']!= -9999]

#remove top outlier 
purchasesFilt = purchasesFilt[ purchasesFilt['alpha']< 200]



purchasesFilt['alpha'].describe()

count    2916.000000
mean       12.290439
std        15.712978
min         0.000000
25%         2.077500
50%         7.210000
75%        16.800000
max       189.740000
Name: alpha, dtype: float64

In [66]:
fig = px.histogram(purchasesFilt, x="alpha",
 height= graphDims['height'] , width= graphDims['width'], 
 nbins=50, 
 marginal="box", 
 title="alpha Distribution")
fig.show()

In [67]:
# alphaSer = purchasesFilt.groupby('senatorId')['alpha'].mean()
alphaSer = purchasesFilt.groupby('senator')['alpha'].mean()
alphaSer =alphaSer.sort_values(ascending=False).head(10)
fig = px.bar(alphaSer,
 height= graphDims['height'] , width= graphDims['width'], 
 title="Top Performing Senators", 
 )


fig.update_layout(yaxis_title="Mean Alpha")
fig.show()

In [68]:
amountSer = purchasesFilt.groupby('amount')['alpha'].mean()
amountSer.index

Index(['$1,000,001 - $5,000,000', '$1,001 - $15,000', '$100,001 - $250,000',
       '$15,001 - $50,000', '$25,000,001 - $50,000,000', '$250,001 - $500,000',
       '$5,000,001 - $25,000,000', '$50,001 - $100,000'],
      dtype='object', name='amount')

In [69]:
amountDict = dict(amountSer)

orderedKeys = ['$25,000,001 - $50,000,000', 
    '$5,000,001 - $25,000,000',
'$1,000,001 - $5,000,000', 
'$250,001 - $500,000',
'$100,001 - $250,000',
'$50,001 - $100,000',
'$15,001 - $50,000',
'$1,001 - $15,000'
]

valuesOrdered =  [amountDict[key] for key in orderedKeys]
amountDict_ordered = dict(zip(orderedKeys, valuesOrdered))
amountDict_ordered

{'$25,000,001 - $50,000,000': 0.59,
 '$5,000,001 - $25,000,000': 12.19,
 '$1,000,001 - $5,000,000': 3.1666666666666665,
 '$250,001 - $500,000': 8.377142857142857,
 '$100,001 - $250,000': 18.626666666666665,
 '$50,001 - $100,000': 13.73,
 '$15,001 - $50,000': 14.358378378378378,
 '$1,001 - $15,000': 11.548437213565535}

In [70]:
amountSer_ordered = pd.Series(amountDict_ordered)
amountFrame_ordered = pd.DataFrame(amountSer_ordered)
amountFrame_ordered.columns =["mean alpha"]

fig = px.bar(amountFrame_ordered)

fig.update_layout(xaxis_title="purchase value", yaxis_title="mean alpha", title="Mean Alpha by Purchase Amount")

fig.show()



In [71]:
countSer = purchasesFilt.groupby('amount')['alpha'].count()
countDict = dict(countSer)
valuesOrdered =  [countDict [key] for key in orderedKeys]
countDict_ordered = dict(zip(orderedKeys, valuesOrdered))


countSer_ordered = pd.Series(countDict_ordered)
countFrame_ordered = pd.DataFrame(countSer_ordered)
countFrame_ordered.columns =["count"]

fig = px.bar(countFrame_ordered)

fig.update_layout(xaxis_title="purchase value", yaxis_title="count", title="Count by Purchase Amount")

fig.show()


In [72]:
senators[senators['senator'].str.contains('Mitch')]

,row,senator,senatorId,party
5,6,"A. Mitchell Mcconnell, Jr.",105,Republican


In [73]:
purchasesFilt[purchasesFilt['senatorId']==105].sort_values(by="transaction_date").head(2)

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
6557,2017-09-07,Spouse,WFC,Wells Fargo & Company,Stock,Purchase,"$1,001 - $15,000",Dividend Reinvestment,"A. Mitchell Mcconnell, Jr.",https://efdsearch.senate.gov/search/view/ptr/0...,05/15/2018,4740,105,17.62
6556,2017-12-05,Spouse,WFC,Wells Fargo & Company,Stock,Purchase,"$1,001 - $15,000",Dividend Reinvestment,"A. Mitchell Mcconnell, Jr.",https://efdsearch.senate.gov/search/view/ptr/0...,05/15/2018,4739,105,6.72


In [74]:
graphAlphaGrowth_fromSource(105, '2017-09-07','WFC')

# Post alpha (from purchases) to db

In [75]:
purchases.tail()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
8388,2014-12-10,Spouse,MDDVX,BlackRock Equity Dividend Inv A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8749,106,0.27
8389,2015-12-11,Spouse,DVFAX,Cohen & Steers Dividend Value A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8750,106,0.43
8390,2014-12-11,Spouse,DVFAX,Cohen & Steers Dividend Value A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8751,106,0.29
8391,2014-12-19,Spouse,BA,The Boeing Company (NYSE),Stock,Purchase,"$1,001 - $15,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/f...,01/05/2015,8758,133,23.90
8392,2014-12-11,Spouse,MIK,"The Michaels Companies, Inc. (NASDAQ)",Stock,Purchase,"$1,001 - $15,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/f...,01/05/2015,8760,133,22.83


In [76]:



def findFullDates(frame):
    frame['deleteMe'] = frame['transaction_date'].astype(str)
    fullDates = frame[frame['deleteMe'].str.len() > 10].shape[0]
    print('fullDates', fullDates)

    frame.drop('deleteMe', axis=1, inplace=True)

findFullDates(purchases)


fullDates 0


In [77]:

findFullDates(transactions)



fullDates 0


In [78]:
transactions.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
0,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,0,100,None
1,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,2,100,None
2,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,3,100,None
3,2022-10-06,Joint,XOM,Exxon Mobil Corporation Common Stock Option Ty...,Stock Option,Sale (Full),"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,8,100,None
4,2022-10-06,Joint,CVX,Chevron Corporation Common Stock Option Type: ...,Stock Option,Sale (Full),"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,9,100,None


In [79]:
t = transactions[transactions['type']!= 'Purchase']
t['alpha'] = np.nan
t.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
0,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,0,100,NaN
1,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,2,100,NaN
2,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,3,100,NaN
3,2022-10-06,Joint,XOM,Exxon Mobil Corporation Common Stock Option Ty...,Stock Option,Sale (Full),"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,8,100,NaN
4,2022-10-06,Joint,CVX,Chevron Corporation Common Stock Option Type: ...,Stock Option,Sale (Full),"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,9,100,NaN


In [80]:
t.type.value_counts()

Sale (Full)       2525
Sale (Partial)    1879
Unknown            467
Exchange           102
Name: type, dtype: int64

In [81]:
print('t Length', t.shape[0])
print('purchases Length', purchases.shape[0])

t Length 4973
purchases Length 3420


In [82]:
transactions_new = pd.concat([t, purchases])
transactions_new.shape


(8393, 14)

In [83]:

findFullDates(transactions_new)

fullDates 0


looks like its interpreting teh date as a date when i concat

In [84]:
transactions_new.tail()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
8388,2014-12-10,Spouse,MDDVX,BlackRock Equity Dividend Inv A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8749,106,0.27
8389,2015-12-11,Spouse,DVFAX,Cohen & Steers Dividend Value A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8750,106,0.43
8390,2014-12-11,Spouse,DVFAX,Cohen & Steers Dividend Value A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8751,106,0.29
8391,2014-12-19,Spouse,BA,The Boeing Company (NYSE),Stock,Purchase,"$1,001 - $15,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/f...,01/05/2015,8758,133,23.90
8392,2014-12-11,Spouse,MIK,"The Michaels Companies, Inc. (NASDAQ)",Stock,Purchase,"$1,001 - $15,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/f...,01/05/2015,8760,133,22.83


In [85]:
timeTestEnd = transactions_new.tail()['transaction_date'].values[0]
timeTestEnd

numpy.datetime64('2014-12-10T00:00:00.000000000')

In [86]:
timeTestStart = transactions_new['transaction_date'][0]
timeTestStart

Timestamp('2022-10-28 00:00:00')

In [87]:
for test in [timeTestStart, timeTestEnd]:
    test = str(test)
    print(test.split(" "))

['2022-10-28', '00:00:00']
['2014-12-10T00:00:00.000000000']


In [88]:


transactions_new['transaction_date'] = transactions_new['transaction_date'].apply(lambda row: extractDateFromTimestamp(row))
transactions_new.tail()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
8388,2014-12-10,Spouse,MDDVX,BlackRock Equity Dividend Inv A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8749,106,0.27
8389,2015-12-11,Spouse,DVFAX,Cohen & Steers Dividend Value A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8750,106,0.43
8390,2014-12-11,Spouse,DVFAX,Cohen & Steers Dividend Value A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/d...,01/09/2015,8751,106,0.29
8391,2014-12-19,Spouse,BA,The Boeing Company (NYSE),Stock,Purchase,"$1,001 - $15,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/f...,01/05/2015,8758,133,23.90
8392,2014-12-11,Spouse,MIK,"The Michaels Companies, Inc. (NASDAQ)",Stock,Purchase,"$1,001 - $15,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/f...,01/05/2015,8760,133,22.83


In [89]:
transactions_new[transactions_new['alpha'].isnull()].type.value_counts()

Sale (Full)       2525
Sale (Partial)    1879
Unknown            467
Exchange           102
Name: type, dtype: int64

In [90]:
transactions_new.alpha.value_counts()

-9999.00    503
 0.00       340
 6.56         7
 7.43         7
 0.16         7
           ... 
 12.87        1
 17.19        1
 15.51        1
 5.40         1
 22.83        1
Name: alpha, Length: 1585, dtype: int64

In [91]:
transactions_new['alpha'].replace({-9999:"Ticker Not Found"}, inplace=True)
numAlpha =  transactions_new[transactions_new['alpha'].notnull()]

numAlpha.alpha.value_counts().values.min(), numAlpha.alpha.value_counts().values.max()


(1, 503)

In [92]:

transactions_new.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha
0,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,0,100,NaN
1,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,2,100,NaN
2,2022-10-28,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,3,100,NaN
3,2022-10-06,Joint,XOM,Exxon Mobil Corporation Common Stock Option Ty...,Stock Option,Sale (Full),"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,8,100,NaN
4,2022-10-06,Joint,CVX,Chevron Corporation Common Stock Option Type: ...,Stock Option,Sale (Full),"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,2022-11-10,9,100,NaN


In [93]:
testParams

(123, '2020-02-13', 'DLR')

In [94]:
# transactions_new[(transactions_new['senatorId']==123) & ]

def findRelevantTransactions_new(senId, date, ticker):
    # startDate = pd.to_datetime(date) + relativedelta(days=-1)
    startDate = pd.to_datetime(date)
    endDate = startDate + relativedelta(years=1)
    
    afterStartDate = transactions_new['transaction_date']>= startDate
    beforeEndDate = transactions_new['transaction_date']< endDate
    isStock = transactions_new['asset_type']=="Stock"
    isSenator = transactions_new['senatorId']==senId
    isTicker = transactions_new['ticker']==ticker
    # rez  = transactions[afterStartDate & beforeEndDate & isStock & isSenator  ]

    rez = transactions_new[afterStartDate & beforeEndDate & isStock & isSenator & isTicker]

    return rez 


# findRelevantTransactions(*testParams)

In [95]:
alphaFrame(*testParams)

,date,DLR,SPY,transactionType,transactionValue,DLR_growth,DLR_pchange,SPY_growth,SPY_pchange,alpha
0,2020-02-07,112.351500,317.990100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,2020-02-13,118.772614,322.420757,Purchase,118.772614,100.000000,0.057152,100.000000,0.013933,0.000000
1,2020-02-14,119.842800,323.159200,NaN,NaN,100.901037,0.009010,100.229031,0.002290,0.672007
2,2020-02-21,125.885100,319.215400,NaN,NaN,105.988321,0.050419,99.005847,-0.012204,6.982475
3,2020-02-28,109.462500,283.587500,NaN,NaN,92.161397,-0.130457,87.955721,-0.111611,4.205676
4,2020-03-06,116.871800,284.736100,NaN,NaN,98.399619,0.067688,88.311963,0.004050,10.087656
5,2020-03-13,130.633300,257.799800,NaN,NaN,109.986044,0.117749,79.957569,-0.094601,30.028475
6,2020-03-20,111.420100,220.358500,NaN,NaN,93.809588,-0.147077,68.345010,-0.145234,25.464578
7,2020-03-27,122.130000,244.070200,NaN,NaN,102.826734,0.096122,75.699283,0.107605,27.127451
8,2020-04-03,125.476300,239.033100,NaN,NaN,105.644134,0.027399,74.137007,-0.020638,31.507127


In [96]:
# transactions_new.to_csv("./CSV Files/transactions_new.csv")

In [97]:

import psycopg2 

conn=None
cur=None 
db = None

def postDataFrameDB():
    try:
        conn_string = f"postgresql://{username}:{pwd}@{host}:{port_id}/{database}"
        db =  create_engine(url=conn_string)
        conn=db.connect()

        transactions_new.to_sql('transactions', con=conn, if_exists='replace', index=False)

        connPsy = psycopg2.connect(conn_string)
        cur = connPsy.cursor()
        connPsy.autocommit = True

        sqlString = """
        select * from transactions limit 5;
        """
        cur.execute(sqlString)

        records = cur.fetchall()
        for row in records:
            print(row)

    except Exception as err:
        print(err)
    finally: 
        if cur is not None: 
            cur.close()
        if conn is not None: 
            conn.close()
    
postDataFrameDB()


('2022-10-28', 'Joint', 'MSFT', 'Microsoft Corporation - Common Stock Option Type: Short Sale Strike price: $245.00  Expires: 06/16/2023 ', 'Stock Option', 'Sale (Full)', '$1,001 - $15,000', '--', 'Thomas H Tuberville', 'https://efdsearch.senate.gov/search/view/ptr/bdd81552-894d-4dc5-b1f3-9fe349529bf8/', '2022-11-10', 0, 100, None)
('2022-10-28', 'Joint', 'MSFT', 'Microsoft Corporation - Common Stock Option Type: Short Sale Strike price: $245.00  Expires: 06/16/2023 ', 'Stock Option', 'Sale (Full)', '$1,001 - $15,000', '--', 'Thomas H Tuberville', 'https://efdsearch.senate.gov/search/view/ptr/bdd81552-894d-4dc5-b1f3-9fe349529bf8/', '2022-11-10', 2, 100, None)
('2022-10-28', 'Joint', 'MSFT', 'Microsoft Corporation - Common Stock Option Type: Short Sale Strike price: $245.00  Expires: 06/16/2023 ', 'Stock Option', 'Sale (Full)', '$1,001 - $15,000', '--', 'Thomas H Tuberville', 'https://efdsearch.senate.gov/search/view/ptr/bdd81552-894d-4dc5-b1f3-9fe349529bf8/', '2022-11-10', 3, 100, None